In [3]:
# Import necessary libraries
import pandas as pd
import itertools

In [4]:
# Import the three created dataframes, and inspect them afterwards
# Yearly ESG Scores and Total Return
df_esg_tr = pd.read_csv("../3) Data/comp_tr_esg_df.csv").rename(columns={"Unnamed: 0": "Year", "key": "ISIN"})

# General Information about the companies (Geogpraphy, Industry, etc.)
df_info = pd.read_csv("../3) Data/comp_info_df.csv").drop(columns=["Unnamed: 0", "Instrument"]
                                               ).rename(columns={"key": "ISIN"})

# Financial Information about the companies (Revenue, ROE, etc.)
df_fin = pd.read_csv("../3) Data/comp_fin_df.csv").rename(columns={"Unnamed: 0": "Year", "key": "ISIN"})

In [5]:
df_esg_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224852 entries, 0 to 224851
Data columns (total 9 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Year                        224852 non-null  int64  
 1   Total Return                162556 non-null  float64
 2   ESG Score                   79820 non-null   float64
 3   ESG Combined Score          79820 non-null   float64
 4   ESG Controversies Score     79811 non-null   float64
 5   Social Pillar Score         79811 non-null   float64
 6   Governance Pillar Score     79820 non-null   float64
 7   Environmental Pillar Score  79811 non-null   float64
 8   ISIN                        224852 non-null  object 
dtypes: float64(7), int64(1), object(1)
memory usage: 15.4+ MB


In [6]:
df_fin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215896 entries, 0 to 215895
Data columns (total 13 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   Year                                   215896 non-null  int64  
 1   Revenue - Mean                         121224 non-null  float64
 2   Employees - Average                    13550 non-null   float64
 3   Earnings Per Share - Mean              123578 non-null  float64
 4   Return On Assets - Mean                78514 non-null   float64
 5   Return On Equity - Mean                103224 non-null  float64
 6   Volume                                 205099 non-null  float64
 7   Company Market Cap                     152548 non-null  float64
 8   Free Cash Flow - Mean                  52839 non-null   float64
 9   Weighted Average Cost of Capital, (%)  65025 non-null   float64
 10  EBITDA - Mean                          105654 non-null  

In [7]:
df_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9766 entries, 0 to 9765
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Country of Headquarters       9760 non-null   object
 1   Region of Headquarters        9760 non-null   object
 2   NAICS National Industry Name  9758 non-null   object
 3   NAICS Industry Group Name     9758 non-null   object
 4   NAICS Sector Name             9758 non-null   object
 5   GICS Sub-Industry Name        9450 non-null   object
 6   GICS Industry Name            9450 non-null   object
 7   ICB Industry name             9733 non-null   object
 8   ICB Sector name               9733 non-null   object
 9   GICS Industry Group Name      9450 non-null   object
 10  SIC Industry Name             2909 non-null   object
 11  TRBC Industry Name            9760 non-null   object
 12  TRBC Industry Group Name      9760 non-null   object
 13  TRBC Business Sect

In [8]:
# Combining them into one, without dropping any values
df_comb = df_esg_tr.merge(df_fin, on=["ISIN", "Year"], how="left"
                          ).merge(df_info, on=["ISIN"], how="left")

# Only keeping relevant columns
df_comb = df_comb[["ISIN",
                   "Year",
                   "Total Return",
                   "ESG Score",
                   "ESG Combined Score",
                   "ESG Controversies Score",
                   "Social Pillar Score",
                   "Governance Pillar Score",
                   "Environmental Pillar Score",
                   "Revenue - Mean",
                   "Earnings Per Share - Mean",
                   "Return On Equity - Mean",
                   "Volume",
                   "Company Market Cap",
                   "EBITDA - Mean",
                   "Country of Headquarters",
                   "Region of Headquarters",
                   "TRBC Industry Name",
                   "TRBC Industry Group Name",
                   "TRBC Business Sector Name",
                   "TRBC Economic Sector Name"]]

# Saving the general dataset
df_comb.to_csv("main_df_with_NAN.csv")

In [9]:
# Deleting NAN values
df_comb_cleaned = df_comb.dropna()

# Saving the cleaned general dataset
df_comb_cleaned.to_csv("main_df_no_NAN.csv")

In [10]:
# We noticed a problem with outliers, so we decided,
# to only keep the middle 99% of numerical datapoints

# Creating a copy of the original dataframe
df_comb_cleaned_99p = df_comb_cleaned.copy()

# Selecting only the float64 columns
float_cols = df_comb_cleaned.select_dtypes(include=['float64']).columns

# Calculating the middle 99% quantiles for each column
quantiles = df_comb_cleaned[float_cols].quantile([0.005, 0.995])

# Keeping only those values
df_comb_cleaned_99p[float_cols] = df_comb_cleaned[float_cols][(df_comb_cleaned[float_cols] >= quantiles.loc[0.005]
                                                               ) & (df_comb_cleaned[float_cols] <= quantiles.loc[0.995])]

# Deleting NAN values
df_comb_cleaned_99p = df_comb_cleaned_99p.dropna()

# Saving the cleaned general dataset that contains
# only the middle 99% of data, to remove outliers
df_comb_cleaned_99p.to_csv("main_df_no_NAN_99p.csv")

In [25]:
# Creating a version that contains YoY changes of
# the numerical columns

# A standardized df is needed, where each ISIN has 
# the same number of years next to it
years = list(range(2000, 2023))
isin = df_comb_cleaned_99p['ISIN'].unique()
combinations = list(itertools.product(isin, years))
df_combinations = pd.DataFrame(combinations, columns=['ISIN', 'Year'])

df_YoY = pd.merge(df_combinations, df_comb_cleaned_99p, on=['ISIN', 'Year'], how='left'
                    ).reset_index().drop(columns="index")

# Creating a list of float64 columns
float_cols = df_YoY.select_dtypes(include=['float64']).columns.tolist()

# Calculating one-period differences: (Xt/Xt-1) - 1
for col in float_cols:
    df_YoY[f'{col} YoY change'] = (df_YoY[col] / df_YoY[col].shift(1)) - 1

# Dropping the first year of every company, because it is compared to the last
# ESG Score of the previous company, not the previous year and already calculated returns
df_YoY = df_YoY[df_YoY["Year"] != 2000].drop(columns=["Total Return YoY change",
                                                            "Return On Equity - Mean YoY change",
                                                            "Volume YoY change"])

# Deleting NAN values
df_YoY_cleaned = df_YoY.dropna()

# Saving this version of the main dataframe
df_YoY_cleaned.to_csv("main_df_no_NAN_99p_YoY.csv")